# galaxyEmulator step-by-step #

### Import the modules

In [1]:
import sys
import os
import numpy as np
from astropy.cosmology import Planck15
import h5py

In [2]:
sys.path.append('..')

In [3]:
from galaxyEmulator.utils import *
from galaxyEmulator.config import *
from galaxyEmulator.prepare import *
from galaxyEmulator.postprocess import *

### Get config and check if conflictions exist

In [4]:
# get default config
config = get_config()

In [5]:
# check if confictions exists in config
# flags > 0 means conflictions exist, 
# Please edit config according to the output message
flags = check_config(config)
print('issue flag: ', flags)

config:
filePath = ../TNG-100
simulationMode = ExtinctionOnly
includeDust = False
minWavelength = 0.01
maxWavelength = 1.2
boxLengthScale = 10
maxBoxLength = 200
wavelengthGrid = Linear
numWavelengths = 500
minLevel = 6
maxLevel = 10
numViews = 3
randomViews = False
inclinations = 0,90,180
azimuths = 0,90,180
resolution = 100
numPackets = 1e7
SEDFamily = BC03
initialMassFunction = Chabrier
FoVboxLengthRatio = 1
minStellarMass = 1
maxStellarMass = inf
saveDatacube = False
workingDir = run
filters = CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y,JWST.F104W
includePSF = True
PSFFromFile = True
includeBackground = False
imgDisplay_CSST = True
RGBImg_CSST = True
RGBFilters_CSST = g,r,z
displayFilter_CSST = r
displaySEDxlogscale_CSST = True
snapNum = 99
fixedRedshift = 0.005
numGeneration = All
numThreads = 12
Found several conflictions in config.ini, please revise!!!
filePath not found
postprocessing path not found
Throughput JWST.F104W not found
use add_filters.py or directly add them 

### Modify config and save it

In [6]:
config['filePath'] = '../../TNG-100'
config['filters'] = 'CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y'
config['includePSF'] = False
config['numGeneration'] = 5
config['saveDatacube'] = True

In [7]:
flags = check_config(config)

config:
filePath = ../../TNG-100
simulationMode = ExtinctionOnly
includeDust = False
minWavelength = 0.01
maxWavelength = 1.2
boxLengthScale = 10
maxBoxLength = 200
wavelengthGrid = Linear
numWavelengths = 500
minLevel = 6
maxLevel = 10
numViews = 3
randomViews = False
inclinations = 0,90,180
azimuths = 0,90,180
resolution = 100
numPackets = 1e7
SEDFamily = BC03
initialMassFunction = Chabrier
FoVboxLengthRatio = 1
minStellarMass = 1
maxStellarMass = inf
saveDatacube = True
workingDir = run
filters = CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y
includePSF = False
PSFFromFile = True
includeBackground = False
imgDisplay_CSST = True
RGBImg_CSST = True
RGBFilters_CSST = g,r,z
displayFilter_CSST = r
displaySEDxlogscale_CSST = True
snapNum = 99
fixedRedshift = 0.005
numGeneration = 5
numThreads = 12


In [8]:
# Now the flag == 0, we can proceed
flags

0

In [16]:
# saving is optional, as we can use the config in memory
# config = save_config(config)

### Load the subhalos and generate the .ski file

In [17]:
# get the snapshot num we want to use
snapnum = np.int32(config['snapNum'])

In [23]:
# get the redshift of current snapshot
snap = h5py.File(os.path.join(config['filePath'], 
                     f'snapdir_{snapnum:03d}/snap_{snapnum:03d}.0.hdf5'), 'r')
snapz = dict(snap['Header'].attrs.items())['Redshift']

In [25]:
# get the subhalos
subhalos = get_subhalos(config['filePath'], snapnum)

In [26]:
# get the min and max stellar mass for subhalos
minStellarMass = np.float32(config['minStellarMass'])
if config['maxStellarMass'] == 'inf':
    maxStellarMass = np.inf
else:
    maxStellarMass = np.float32(config['maxStellarMass'])

In [28]:
# get the SubhaloIDs following the stellar mass condition 
stellarMass = subhalos['SubhaloMassType'][:, 4] / Planck15.h
    
subhalo_indices = np.where((stellarMass > minStellarMass) \
                        & (stellarMass < maxStellarMass) \
                        & (subhalos['SubhaloFlag'] == 1) \
                        & (subhalos['SubhaloParent'] == 0))[0]

subhaloNums = subhalo_indices.shape[0]
print(f'Number of subhalos is {subhaloNums} in snapshot {snapnum} in stellar mass range {minStellarMass} to {maxStellarMass} [10^10 M_sun]')

Number of subhalos is 6145 in snapshot 99 in stellar mass range 1.0 to inf [10^10 M_sun]


In [9]:
# Let us generate the galaxy from subhalo with ID = 1
subhaloID = 1

In [36]:
print(f'Stellar Mass: {stellarMass[subhaloID]} [10^10 M_sun]')

Stellar Mass: 55.87917110509566 [10^10 M_sun]


In [41]:
boxLength = subhalos['SubhaloHalfmassRadType'][:, 4][subhaloID] \
        * np.float32(config['boxLengthScale']) / Planck15.h

boxLength = np.min([boxLength, np.float32(config['maxBoxLength'])])
print('boxLength in kpc: ', np.around(boxLength, 2))

boxLength in kpc:  200.0


In [42]:
# create the working directory for SKIRT
workingDir = config['workingDir']
os.makedirs(workingDir, exist_ok=True)

In [43]:
# get and save the star-forming, quenched or dust particles
particles_from_tng(subhaloID, snapnum, snapz, subhalos, boxLength, config)

Star-forming star particles: 1
Quenched star particles: 511871


In [44]:
# create .ski file for SKIRT
fixedRedshift = np.float32(config['fixedRedshift'])
property = modify_ski_file(fixedRedshift, boxLength, config)

### run SKIRT

In [45]:
# run SKIRT, this will take some time and memory
run_skirt(config)

27/08/2024 08:35:37.983   Welcome to SKIRT v9.0 (git 382ad4b built on 15/08/2024 at 01:46:45)
27/08/2024 08:35:37.983   Running on 2488h for xczhou
27/08/2024 08:35:37.983   Constructing a simulation from ski file 'skirt.ski'...
27/08/2024 08:35:38.042   Starting simulation skirt using 12 threads and a single process...
27/08/2024 08:35:38.042   Starting setup...
27/08/2024 08:35:38.043     Panchromatic wavelength regime
27/08/2024 08:35:38.043     With transfer medium
27/08/2024 08:35:38.043     Redshift: 0.005
27/08/2024 08:35:38.043     Luminosity distance: 22.2131 Mpc
27/08/2024 08:35:38.043     Model and grid symmetry: 3D
27/08/2024 08:35:38.043     Photon life cycle: no explicit absorption; with forced scattering
27/08/2024 08:35:38.043   BruzualCharlotSEDFamily opened stored table /share/xczhou/SKIRT/resources/SKIRT9_Resources_Core/SED/BruzualCharlotSEDFamily_Chabrier_hr.stab
27/08/2024 08:35:38.044   ParticleSource reads smoothed source particles from text file /share/xczhou/il

In [10]:
property = {'redshift': np.float32(0.005),
 'lumiDis': np.float64(22.213014431289825),
 'inclinations': [0.0, 90.0, 180.0],
 'azimuths': [0.0, 90.0, 180.0],
 'FoV': np.float64(200000.0)}

### postprocessing for calculating bandpass images, save them and clean the workingDir

In [11]:
postprocess(subhaloID, property, config)

KeyError: 'saveDataCube'